In [ ]:
from datasets import load_dataset
import pandas as pd
import numpy as np

# Installing the dataset

User Reviews:

In [ ]:
dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_Grocery_and_Gourmet_Food", trust_remote_code=True)
print(dataset['full'][0])

In [ ]:
df_reviews = dataset['full'].to_pandas()

In [ ]:
df_reviews.head(5)

In [ ]:
df_reviews.shape

Item Metadata:

In [ ]:
dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_Grocery_and_Gourmet_Food", trust_remote_code=True)
print(dataset['full'][0])

In [ ]:
df_meta = dataset['full'].to_pandas()

In [ ]:
df_meta.head(5)

In [ ]:
df_meta.shape

# Drop unnecessary columns

In [ ]:
df_reviews = df_reviews.drop(columns=['images'], axis=1)
df_reviews

In [ ]:
df_meta = df_meta.drop(columns=['images'], axis=1)
df_meta

In [ ]:
df_meta = df_meta.drop(columns=['videos'], axis=1)
df_meta

# Find items using categories & parent_asin

In [ ]:
df_meta['categories'] = df_meta['categories'].apply(lambda x: tuple(x) if isinstance(x, (list, np.ndarray)) else x)
df_meta[['categories', 'parent_asin']].drop_duplicates()

In [ ]:
df_carb = df_meta[df_meta['title'].str.contains('low carb', case=False, na=False)]
parent_asin_list_carb = df_carb['parent_asin'].drop_duplicates().to_list()
len(parent_asin_list_carb)

In [ ]:
df_carb.head(5)

In [ ]:
df_snacks = df_meta[df_meta['title'].str.contains('snack', case=False, na=False)]
parent_asin_list_snacks = df_snacks['parent_asin'].drop_duplicates().to_list()
len(parent_asin_list_snacks)

In [ ]:
df_snacks.head(5)

In [ ]:
parent_asin_list = parent_asin_list_carb + parent_asin_list_snacks
df_healthy = df_reviews[df_reviews['parent_asin'].isin(parent_asin_list)]
df_healthy.head(5)

In [ ]:
df_meta = df_meta.rename(columns={'parent_asin': 'parent_asin_meta'})
joined_df = pd.merge(df_healthy, df_meta, left_on="parent_asin", right_on="parent_asin_meta", how="left")
joined_df.head(10)

# Remove stop words and tokenize text

In [ ]:
import nltk
import spacy
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess
import string

In [ ]:
nltk.download('stopwords')
stop_words = stopwords.words('english')

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
def preprocess_text(text):
    # Tokenize and remove punctuation
    tokens = simple_preprocess(text, deacc=True)

    # Remove stopwords and punctuation
    tokens = [word for word in tokens if word not in stop_words and word not in string.punctuation]
    
    return tokens

In [ ]:
joined_df['cleaned_text'] = joined_df['text'].apply(preprocess_text)
joined_df.head(10)

In [ ]:
processed_reviews = joined_df['cleaned_text'].tolist()

# Train the LDA model

In [ ]:
from tqdm import tqdm
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore
import logging

In [ ]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
if isinstance(joined_df['cleaned_text'].iloc[0], list):
    processed_reviews = joined_df['cleaned_text'].tolist()
else:
    raise ValueError("Ensure 'processed_reviews' contains lists of tokens")

In [ ]:
dictionary = corpora.Dictionary(processed_reviews)

In [ ]:
corpus = [dictionary.doc2bow(review) for review in processed_reviews]
total_docs = len(corpus)

In [ ]:
def custom_lda_training(corpus, dictionary, num_topics, passes, chunksize):
    num_iterations = passes

    with tqdm(total=total_docs * num_iterations, desc="LDA Training Progress") as pbar:
        # Initialize the LDA Multicore model
        lda_model = LdaMulticore(
            corpus=corpus,
            id2word=dictionary,
            num_topics=num_topics,
            random_state=100,
            chunksize=chunksize,
            passes=1,
            workers=1,
            alpha='symmetric',
            per_word_topics=True,
            batch=True
        )

        # Manually iterate over the passes and update the progress bar
        for pass_ in range(num_iterations):
            for i in range(0, total_docs, chunksize):
                chunk = corpus[i:i + chunksize]
                lda_model.update(chunk)
                pbar.update(len(chunk))

    return lda_model

In [ ]:
lda_model = custom_lda_training(corpus, dictionary, num_topics=20, passes=5, chunksize=3000)